In [31]:
!pip install tqdm
!pip install torch-summary
!pip install -U scikit-learn
!pip install tensorboard


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 49.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 74.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 ->

In [32]:
SHOULD_PRINT = True
SEED = 32
CONTINUE_MODEL = None

In [9]:
import torch
from tqdm import tqdm

torch.manual_seed(SEED)

In [10]:
# from torch.utils.data import Dataset

# class CustomDataset(Dataset):
#     def __init__(self, filename):
#         self.data = []
#         with open(filename, 'r') as f:
#             for line in f:
#                 numbers = [float(num) for num in line.strip().split()]
#                 self.data.append((numbers[:4], numbers[4:]))

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         inputs, targets = self.data[idx]
#         return torch.tensor(inputs), torch.tensor(targets)

In [11]:
# import torch

# from typing import List
# from torch.utils.data import Dataset


# class CustomDataset(Dataset):
#     def __init__(self, large_file_path, chunk_size):
#         self.large_file_path = large_file_path
#         self.line_offsets = self.get_line_offsets(large_file_path, chunk_size)

#     def get_line_offsets(self, path: str, chunk_size: int) -> List[int]:
#         offsets = [0]
#         with open(path, "rb") as file:
#             chunk = file.readlines(chunk_size)
#             while chunk:
#                 for line in chunk:
#                     offsets.append(offsets[-1] + len(line))
#                 chunk = file.readlines(chunk_size)
#                 print(f"Lines found: {len(offsets)}", end='\r')
#         offsets = offsets[:-1]
#         return offsets
    
#     def __len__(self):
#         return len(self.line_offsets)
    
#     def __getitem__(self, line):
#         offset = self.line_offsets[line]
#         with open(self.large_file_path, 'r', encoding='utf-8') as f:
#             f.seek(offset)
#             line = f.readline()
#             numbers = [float(num) for num in line.strip().split()]
#             inputs, targets = numbers[:4], numbers[4:]
#             return torch.tensor(inputs), torch.tensor(targets)

In [12]:
import numpy as np

features = [[1, 2, 3, 4], [2, 3, 4, 5]]
mean = np.mean(features, axis=0)
mean

array([1.5, 2.5, 3.5, 4.5])

In [13]:
import torch
import numpy as np
from typing import List
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import random

class CustomDataset(Dataset):
    def __init__(self, large_file_path, chunk_size, subset_size=50000):
        self.large_file_path = large_file_path
        self.chunk_size = chunk_size
        self.subset_size = subset_size
        self.line_offsets = self.get_line_offsets(large_file_path, chunk_size)
        self.scaler = StandardScaler()
        print("Calculating mean and std...")
        self.mean, self.std = self.calculate_mean_std()
        print(f"Mean: {self.mean}, Std: {self.std}")

    def get_line_offsets(self, path: str, chunk_size: int) -> List[int]:
        offsets = [0]
        with open(path, "rb") as file:
            chunk = file.readlines(chunk_size)
            while chunk:
                for line in chunk:
                    offsets.append(offsets[-1] + len(line))
                chunk = file.readlines(chunk_size)
                print(f"Lines found: {len(offsets)}", end='\r')
        offsets = offsets[:-1]
        print(f"Lines found: {len(offsets)}", end='\n')
        return offsets

    def calculate_mean_std(self):
        selected_offsets = random.sample(self.line_offsets, min(self.subset_size, len(self.line_offsets)))
        features = []
        for offset in selected_offsets:
            with open(self.large_file_path, 'r', encoding='utf-8') as f:
                f.seek(offset)
                line = f.readline()
                numbers = [float(num) for num in line.strip().split()]
                features.append(numbers[:4])
        features = np.array(features)
        mean = np.mean(features, axis=0, dtype=np.float32)
        std = np.std(features, axis=0, dtype=np.float32)
        return mean, std

    def standardize_features(self, features):
        standardized_features = (features - self.mean) / self.std
        return standardized_features

    def __len__(self):
        return len(self.line_offsets)

    def __getitem__(self, line):
        offset = self.line_offsets[line]
        with open(self.large_file_path, 'r', encoding='utf-8') as f:
            f.seek(offset)
            line = f.readline()
            numbers = [float(num) for num in line.strip().split()]
            features, targets = numbers[:4], numbers[4:]
            standardized_features = self.standardize_features(np.array(features))
            return torch.tensor(standardized_features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)


In [16]:
filename = "full_dataset.txt"
full_dataset = CustomDataset(filename, 2**20)

Lines found: 6367855
Calculating mean and std...
Mean: [ 2.3069660e+01 -4.5200000e-03  1.5650475e+00  3.1191590e+00], Std: [ 8.339648   14.216207    0.92338675  1.8202546 ]


In [17]:
train_size = int(0.1 * len(full_dataset))
rest_size = len(full_dataset) - train_size
val_size = rest_size // 10
test_size = rest_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(SEED))

In [18]:
train_size

636785

In [19]:
from torch.utils.data import DataLoader

BATCH_SIZE = 2 ** 20
train_shuffle = True
val_shuffle = False

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=train_shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=val_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [20]:
# for inputs, targets in train_dataloader:
#     assert inputs.shape[1] == 4 and targets.shape[1] == 2

In [21]:
import torch.nn as nn
import torch.nn.init as init

In [22]:
class Block(nn.Module):
    def __init__(self, in_layers, out_layers):
        super(Block, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_layers, out_layers),
            nn.ReLU(),
            nn.Dropout1d(p=0.1),
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [23]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.blocks = nn.ModuleList([
            Block(4, 16),
            Block(16, 16),
            Block(16, 16),
        ])
        self.out = nn.Linear(16, 2)
        
        self.__init_weights()

    def __init_weights(self):
        layers = [self.blocks]
        # Initialize linear layers using Kaiming (He) uniform initialization
        for m in layers:
            for layer in m:
                self.__init_layer(layer)
        self.__init_layer(self.out)
                        
    def __init_layer(self, layer):
        if isinstance(layer, nn.Linear):
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='tanh')
            if layer.bias is not None:
                init.zeros_(layer.bias)

    def forward(self, x):
        x = self.blocks[0](x)
        for block in self.blocks[1:]:
            y = block(x)
            x = x + y
        x = self.out(x)
        return x

In [54]:
LR = 1e-3
NUM_EPOCHS = 12
WEIGHT_DECAY = 0.99
WEIGHT_DECAY_L1 = 1e-4
MOMENTUM = 0.9

In [55]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

In [56]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

last_epoch = 0
model = MLP()
loss_mse = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, nesterov=True)
# optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# scheduler = lr_scheduler.LinearLR(
#     optimizer,
#     start_factor=1.0,
#     end_factor=0.2,
#     total_iters=NUM_EPOCHS * 0.75)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=300, verbose=True)
if CONTINUE_MODEL:
    checkpoint = torch.load("checkpoint.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    last_epoch = checkpoint['epoch']
    loss_mse = checkpoint['loss_mse']

Epoch 00000: adjusting learning rate of group 0 to 1.0000e-03.


In [57]:
# from torchsummary import summary

# summary(model, (BATCH_SIZE, 4))

In [58]:
from torch.utils.tensorboard import SummaryWriter

def train(model, dataloader, optimizer, scheduler, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0
    best_diff = 100

    logs_steps = max(int(log_perc * len(dataloader)), 1)
    start_step = epoch * len(dataloader)

    before_lr = optimizer.param_groups[0]["lr"]
    writer.add_scalar('Lr/Train', before_lr, epoch)
    for idx, (inputs, targets) in enumerate(tqdm(dataloader)):
        inputs, targets = inputs.to(device).to(torch.float32), targets.to(device).to(torch.float32)
        
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        diff = torch.abs(outputs - targets).mean()

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Train', diff.item(), start_step + idx)
            
            if SHOULD_PRINT:
                print(f"Loss/Train: {loss.item()}")
                print(f"Absolute Difference/Train: {diff.item()}")
        
    after_lr = optimizer.param_groups[0]["lr"]
    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)
    
    writer.add_scalar('Avg Loss/Train', average_loss, epoch)
    writer.add_scalar('Avg Absolute Difference/Train', average_diff, epoch)
    writer.add_scalar('Lr/Train', after_lr, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Train: {average_loss}")
        print(f"Avg Absolute Difference/Train: {average_diff}")
        print(f"Lr/Train: {after_lr}")

    # print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {average_loss:.4f}, Train Diff: {average_diff:.15f}")

In [59]:
def validate(model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device).to(torch.float32), targets.to(device).to(torch.float32)
            outputs = model(inputs)

            loss = loss_fn(outputs, targets)
            diff = torch.abs(outputs - targets).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg Loss/Val', average_loss, epoch)
        writer.add_scalar('Avg Absolute Difference/Val', average_diff, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Val: {average_loss}")
        print(f"Avg Absolute Difference/Val: {average_diff}")

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Test Diff: {average_diff:.15f}")
        
    return average_diff

In [60]:
def total_loss(outputs, targets):
    loss = loss_mse(outputs, targets)
#     loss_rmse = torch.sqrt(loss)
#     l1_reg = torch.tensor(0., requires_grad=True)

#     for name, param in model.named_parameters():
#         if 'weight' in name:
#             l1_reg = l1_reg + torch.linalg.norm(param, 1)

#     total_loss = (loss * 0.8 + loss_rmse * 0.2) + WEIGHT_DECAY_L1 * l1_reg
#     total_loss = (loss * 0.8 + loss_rmse * 0.2)
    return loss

In [61]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/{now}")

VALIDATION_STEPS = 3

model = model.to(device)
best_avg_diff = 1000

for idx, epoch in enumerate(range(last_epoch, NUM_EPOCHS)):
    train(model, train_dataloader, optimizer, scheduler, total_loss, epoch, writer, log_perc=0.2)

    if idx % VALIDATION_STEPS == 0:
        average_diff = validate(model, val_dataloader, total_loss, epoch, writer)
        if average_diff < best_avg_diff:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss_mse': loss_mse,
                }, "checkpoint.pth")

# Launch TensorBoard: `tensorboard --logdir=tb_logs`

100%|██████████| 1/1 [00:38<00:00, 38.20s/it]

Epoch 00001: adjusting learning rate of group 0 to 9.9997e-04.
Loss/Train: 1.1721032857894897
Absolute Difference/Train: 0.8564786911010742
Avg Loss/Train: 1.1721032857894897
Avg Absolute Difference/Train: 0.8564786911010742
Lr/Train: 0.000999972584682756


Avg Loss/Val: 0.9965173602104187
Avg Absolute Difference/Val: 0.8104553818702698
Epoch [1/12] Val Loss: 0.9965, Val Diff: 0.810455381870270


100%|██████████| 1/1 [00:35<00:00, 35.73s/it]


Epoch 00002: adjusting learning rate of group 0 to 9.9989e-04.
Loss/Train: 1.110743522644043
Absolute Difference/Train: 0.8327600955963135
Avg Loss/Train: 1.110743522644043
Avg Absolute Difference/Train: 0.8327600955963135
Lr/Train: 0.0009998903417374227


...

Avg Loss/Val: 0.3222880959510803
Avg Absolute Difference/Val: 0.45400914549827576
Epoch [10/12] Val Loss: 0.3223, Val Diff: 0.454009145498276


100%|██████████| 1/1 [00:36<00:00, 36.04s/it]


Epoch 00011: adjusting learning rate of group 0 to 9.9669e-04.
Loss/Train: 0.3666348457336426
Absolute Difference/Train: 0.472565233707428
Avg Loss/Train: 0.3666348457336426
Avg Absolute Difference/Train: 0.472565233707428
Lr/Train: 0.000996686382800198


100%|██████████| 1/1 [00:35<00:00, 35.43s/it]

Epoch 00012: adjusting learning rate of group 0 to 9.9606e-04.
Loss/Train: 0.31753915548324585
Absolute Difference/Train: 0.43969660997390747
Avg Loss/Train: 0.31753915548324585
Avg Absolute Difference/Train: 0.43969660997390747
Lr/Train: 0.000996057350657239


In [ ]:
validate(model, test_dataloader, total_loss, epoch, None)